In [1]:
example = "..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3.."
example2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
example3 = "659.1.28.1...5..3.2..8...1....135.7.8..9....2..3.7864.3.2..9..4.....18....876...."
example4 = "85...24..72......9..4.........1.7..23.5...9...4...........8..7..17..........36.4."

In [2]:
def cross(A,B):
    return [a+b for a in A for b in B]

In [3]:
def setUp(puzzle):
    """ add in checks about lengths etc """
    rows = 'ABCDEFGHI'
    columns = '123456789'
    squares = cross(rows, columns)
    grid = dict(zip(squares, puzzle))
    return grid

In [4]:
def setUpDomains(puzzle):
    """input is string of dots and digits
        sets up dict where key is squares, values are domains """
    digits = '123456789'
    grid = setUp(puzzle)
    for a, b in grid.items():
        if b == '.':
            grid[a] = list(digits)
        else:
            grid[a] = list(grid[a])
            
            
    
    #domains = { a : list(grid[a]) for (a, b) in grid.items()}
    
    return grid

In [5]:
def sudoku(puzzle):
    
    # add in checks to ensure set up is ok, e.g. 81 characters long
    
    rows = 'ABCDEFGHI'
    columns = '123456789'
    digits = '123456789'
    
    variables = cross(rows, columns)
    domains = dict(zip(variables, puzzle))
    
    for key, values in domains.items():
        if values == '.':
            domains[key] = list(digits)
        else:
            domains[key] = list(domains[key])
     
    
    
    unitlist = [cross(rows,a) for a in columns] + [cross(a, columns) for a in rows] + [cross(a,b) for a in ['ABC', 'DEF', 'GHI'] for b in ['123', '456', '789']]
    
    units = dict((s,[t for t in unitlist if s in t]) for s in variables)
    
    neighbours =  dict((s, set(units[s][0] +units[s][1]+ units[s][2]) - set([s])) for s in variables)
    
    constraints = lambda X, x, Y, y : x != y
    
    return variables, domains, neighbours, constraints

In [6]:
class CSP():
    def __init__(self, variables, domains, neighbours, constraints):
        self.variables = variables #list of variables
        self.domains = domains #dictionary where keys are variables
        self.neighbours = neighbours #dict: keys are variables, values are lists other variables in constraints with key
        self.constraints = constraints #function X,x,Y,y returns true if X = x, Y = y satisfy constraints
                                        #e.g. lambda X,x,Y,y : x != y
                        
    def prune(self, var, value):
        """ romove value val from domain of variable var """
        self.domains[var].remove(value)
        
    def assign(self, var, val, assignment):
        """ add variable var and value val to assigment """
        assignment[var] = val
        
    def unassign(self, var, assignment):
        """ remove variable var from assignment """
        if var in assignment:
            del assignment[var]
           
    def nConflicts(self, var, val, assignment):
        """ return number of conflicts var = val has in assignent """
        count = 0
        for var2 in self.neighbours[var]:
            if var2 in assignment and not self.constraints(var, val, var2, assignment[var2]):
                count += 1
        return count
    
    def isGoalState(self, assignment):
        """ check is the current state a goal state """
    
        result = (len(assignment) == len(self.variables)) and (all(self.nConflicts(var, assignment[var], assignment) == 0 for var in self.variables))
        return result

In [7]:
def backtracking_search(csp):

    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment #if len(assignment) == 81, return assignment. Do goal test. If goal return True. Else assertion error
        
        #now if len(assignment) < 81, i.e. partial assignment enter for loop.
        
        var = next(iter([v for v in csp.variables if v not in assignment]), None) #var = next variable from variables list that isn't in assignment
        
        for value in csp.domains[var]:  #cycle through remaining domain vales for var
            if 0 == csp.nConflicts(var, value, assignment):
                csp.assign(var, value, assignment)
                result = backtrack(assignment) #runs new backtrack with new var and val. don't get confused between these 
                if result is not None:
                    return result
        csp.unassign(var, assignment) #unassigns previously assigned variable
        return None

    result = backtrack({})
    assert result is None or csp.isGoalState(result) #true if:
    return result

In [8]:
def revise(csp, Xi, Xj):
    revised = False
    for x in csp.domains[Xi]:
        if all(not csp.constraints(Xi,x,Xj,y) for y in csp.domains[Xj]):
            csp.prune(Xi, x) 
            revised = True
    return revised

In [9]:
def AC3(csp):
    queue = {(Xi, Xj) for Xi in csp.variables for Xj in csp.neighbours[Xi]}
    while queue:
        (Xi, Xj) = queue.pop()
        if revise(csp, Xi, Xj):
            if not csp.domains[Xi]:
                return False
            for Xk in csp.neighbours[Xi]:
                if Xk != Xj:
                    queue.add((Xk, Xi))
    return True
                
            
    

In [17]:
variables, domains, neighbours, constraints = sudoku(example4)

In [18]:
problem = CSP(variables, domains, neighbours, constraints)

In [19]:
test = AC3(problem)

In [20]:
domains

{'A1': ['8'],
 'A2': ['5'],
 'A3': ['1', '3', '6', '9'],
 'A4': ['3', '6', '7', '9'],
 'A5': ['1', '6', '7', '9'],
 'A6': ['2'],
 'A7': ['4'],
 'A8': ['1', '3', '6'],
 'A9': ['1', '3', '6', '7'],
 'B1': ['7'],
 'B2': ['2'],
 'B3': ['1', '3', '6'],
 'B4': ['3', '4', '5', '6', '8'],
 'B5': ['1', '4', '5', '6'],
 'B6': ['1', '3', '4', '5', '8'],
 'B7': ['1', '3', '5', '6', '8'],
 'B8': ['1', '3', '5', '6', '8'],
 'B9': ['9'],
 'C1': ['1', '6', '9'],
 'C2': ['3', '6', '9'],
 'C3': ['4'],
 'C4': ['3', '5', '6', '7', '8', '9'],
 'C5': ['1', '5', '6', '7', '9'],
 'C6': ['1', '3', '5', '8', '9'],
 'C7': ['1', '2', '3', '5', '6', '7', '8'],
 'C8': ['1', '2', '3', '5', '6', '8'],
 'C9': ['1', '3', '5', '6', '7', '8'],
 'D1': ['6', '9'],
 'D2': ['6', '8', '9'],
 'D3': ['6', '8', '9'],
 'D4': ['1'],
 'D5': ['4', '5', '6', '9'],
 'D6': ['7'],
 'D7': ['3', '5', '6', '8'],
 'D8': ['3', '5', '6', '8'],
 'D9': ['2'],
 'E1': ['3'],
 'E2': ['6', '7', '8'],
 'E3': ['5'],
 'E4': ['2', '4', '6', '8'],
 'E5'

In [21]:
problemRed = CSP(variables, domains, neighbours, constraints)

In [22]:
solution = backtracking_search(problemRed)

In [23]:
solution

{'A1': '8',
 'A2': '5',
 'A3': '9',
 'A4': '6',
 'A5': '1',
 'A6': '2',
 'A7': '4',
 'A8': '3',
 'A9': '7',
 'B1': '7',
 'B2': '2',
 'B3': '3',
 'B4': '8',
 'B5': '5',
 'B6': '4',
 'B7': '1',
 'B8': '6',
 'B9': '9',
 'C1': '1',
 'C2': '6',
 'C3': '4',
 'C4': '3',
 'C5': '7',
 'C6': '9',
 'C7': '5',
 'C8': '2',
 'C9': '8',
 'D1': '9',
 'D2': '8',
 'D3': '6',
 'D4': '1',
 'D5': '4',
 'D6': '7',
 'D7': '3',
 'D8': '5',
 'D9': '2',
 'E1': '3',
 'E2': '7',
 'E3': '5',
 'E4': '2',
 'E5': '6',
 'E6': '8',
 'E7': '9',
 'E8': '1',
 'E9': '4',
 'F1': '2',
 'F2': '4',
 'F3': '1',
 'F4': '5',
 'F5': '9',
 'F6': '3',
 'F7': '7',
 'F8': '8',
 'F9': '6',
 'G1': '4',
 'G2': '3',
 'G3': '2',
 'G4': '9',
 'G5': '8',
 'G6': '1',
 'G7': '6',
 'G8': '7',
 'G9': '5',
 'H1': '6',
 'H2': '1',
 'H3': '7',
 'H4': '4',
 'H5': '2',
 'H6': '5',
 'H7': '8',
 'H8': '9',
 'H9': '3',
 'I1': '5',
 'I2': '9',
 'I3': '8',
 'I4': '7',
 'I5': '3',
 'I6': '6',
 'I7': '2',
 'I8': '4',
 'I9': '1'}